# DNA and Protein Sequence Analysis Functions

# Primers and melting temperature

### Reading a DNA sequence from a file

The readDNAsequence function is designed to open a FASTA file and skip the header (the line that starts with >). It reads the remaining lines, replacing any U characters with T to handle RNA-to-DNA conversion, and checks if the sequence contains any invalid DNA characters (other than A, C, T, or G). If an invalid character is found, a custom BadSequenceException is raised. The function returns the cleaned DNA sequence as a string. The test code ensures that this function works correctly by writing a sample sequence to a file, reading it back, and checking the output.

In [1]:
import os
import os.path

In [2]:
class BadSequenceException(Exception):
    pass

In [3]:
def readDNAsequence(file_name):
    # Initialize an empty list to store the DNA sequence lines.
    sequence_lines = []

    with open(file_name, 'r') as FASTA:
        skip_header = True  # Flag to skip the header line.
        for line in FASTA: # Iterate through each line in the FASTA file.
            if skip_header:
                skip_header = False
                continue  # Skip the header line and move to the next.

            
            line = line.strip().replace('U', 'T')

        
            if any(base not in 'ACTG' for base in line):
                
                raise BadSequenceException("Wrong characters in the sequence.")

            # Append the processed line to the sequence_lines list.
            sequence_lines.append(line)

    # Join the sequence lines into a single string and return it.
    return ''.join(sequence_lines)


In [4]:
# Create a sample DNA FASTA file
with open("dna_sequence.fas", "wt") as fasta_file:
    fasta_file.write("> Example DNA Sequence\n")
    fasta_file.write("ACUG\n")

# Test the readDNAsequence function
_seq = readDNAsequence("dna_sequence.fas")
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print(f"Sequence: {_seq}")

Sequence: ACTG


### Computing the complement of a sequence
The complement function takes a DNA sequence and returns its complementary sequence by mapping each base (A, T, C, G) to its complement (T, A, G, C). If the sequence contains invalid bases, it raises a BadSequenceException. The test code verifies that the function correctly computes the complement of a given sequence.

In [5]:
def complement(dna_seq):
     # Define a dictionary to map DNA bases to their complementary bases.
    complement_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}

    if any(base not in 'ACTG' for base in dna_seq):
        raise BadSequenceException("Invalid DNA sequence")

    # Construct the complementary sequence by mapping each base to its complement.
    compseq = ''.join(complement_dict[base] for base in dna_seq) 

    return compseq

In [6]:
# Test code
_seq=complement("ACGTTTCG")
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print(f"Complement: {_seq}")

Complement: TGCAAAGC


### Extracting primers
The primer function extracts either a forward or reverse primer from a DNA sequence. If the forward argument is True, it returns the first n bases of the sequence as the forward primer. If forward is False, it reverses the sequence, computes the complement, and returns the first n bases of the reverse complement. If the sequence is too short, the function raises a BadSequenceException. The test code checks that the function correctly returns a reverse primer for a given sequence.

In [7]:
def primer(sequence, length=20, forward=True):
    if len(sequence) < length:
        raise BadSequenceException("Input sequence is too short to generate a primer")

    if forward:
        return sequence[:length]
    else:
        # Reverse the input sequence.
        reverse_sequence = sequence[::-1]
        complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
        # Generate the reverse primer by mapping and joining the complement of the reversed sequence.
        reverse_primer = "".join(complement.get(base, base) for base in reverse_sequence[:length])
        return reverse_primer

In [8]:
# Test code
_seq=primer(sequence="AAAAATTTTTCCCCCGGGGGAAAAA", length= 20, forward=False)
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print(f"Reverse Primer: {_seq}")

Reverse Primer: TTTTTCCCCCGGGGGAAAAA


### Computing the melting temperature
The meltingTemp function calculates the melting temperature (Tm) of a DNA primer using the formula Tm = 4(G + C) + 2(A + T). This formula counts the number of G and C bases, and A and T bases, in the sequence, and applies the respective coefficients to compute Tm. If invalid bases are found in the sequence, a BadSequenceException is raised. The test ensures that the function computes the melting temperature correctly.

In [9]:
def meltingTemp(primer):
    # Define a string of valid DNA bases.
    valid_bases = 'ACTG'
    
    # Check if all characters in the 'primer' string are valid DNA bases (A, C, T, or G).
    if not all(base in valid_bases for base in primer):
        
        raise BadSequenceException("Invalid sequence: Sequence contains characters other than A, C, T, G.")
    
    # Count the occurrences of each DNA base in the 'primer' string.
    G = primer.count('G')
    C = primer.count('C')
    A = primer.count('A')
    T = primer.count('T')
    
    # Calculate the melting temperature (Tm) using the formula: Tm = 4(G + C) + 2(A + T)
    Tm = 4 * (G + C) + 2 * (A + T)
    
    return Tm

In [10]:
# Test code
_temp=meltingTemp("AAAAATTTTTCCCCCGGGGG")
assert ((type(_temp) is type(0.0)) or
        (type(_temp) is type(0))), "Return value is not a number: %r" % _temp
print(f"Melting Temperature: {_temp}")

Melting Temperature: 60


### Putting it all together
The sequencePCRtemp function reads a DNA sequence from a FASTA file, extracts both the forward and reverse primers, calculates their melting temperatures, and returns the average melting temperature. The test code verifies that the function computes the correct average melting temperature using a sample sequence.

In [11]:
def sequencePCRtemp(fasta_file):
    # Read the DNA sequence from the file
    sequence = readDNAsequence(fasta_file)

    # Calculate the forward primer and reverse primer
    forward_primer = primer(sequence)
    reverse_primer = primer(sequence, forward=False)

    # Calculate the melting temperatures for the primers
    forward_temp = meltingTemp(forward_primer)
    reverse_temp = meltingTemp(reverse_primer)

    # Calculate the average melting temperature
    avg_temp = (forward_temp + reverse_temp) / 2

    return avg_temp

In [12]:
# Create a sample DNA FASTA file
with open("pcr_sequence.fas", "wt") as fasta_file:
    fasta_file.write("> Example PCR Sequence\n")
    fasta_file.write("AAAAACCCCCTTTTTGGGGGAAAAA\n")

# Test the sequencePCRtemp function
_temp = sequencePCRtemp("pcr_sequence.fas")
assert type(_temp) is type(0.0), "Return value is not a float: %r" % _temp
print(f"Average Melting Temperature: {_temp}")

Average Melting Temperature: 60.0


# Section 2: Translation and reading frames

### Translating a DNA Sequence in All Reading Frames
The translate function translates a DNA sequence into protein sequences for all six reading frames (three forward frames and three reverse frames). It uses a codon table to convert DNA triplets into amino acids. For the reverse frames, the function reverses the sequence before translating. The test code verifies that the function outputs a dictionary with correctly translated sequences for all six reading frames.

In [13]:
def translate(DNA_sequence):
  # Define a codon table for DNA to protein translation.
    codon_table = {
        'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
        'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
        'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
        'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
        'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
        'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
        'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
        'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
        'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
        'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
        'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
        'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
        'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
        'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
        'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
        'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G'
    }

    # Create a helper function to translate a single frame
    def translate_frame(sequence):
      # Translate a DNA sequence into a protein sequence for a single frame.
        translation = ''.join([codon_table[sequence[i:i+3]] for i in range(0, len(sequence), 3) if i+3 <= len(sequence)])
        return translation

    frames = {}
    # Translate three forward frames.
    frames['f1'] = translate_frame(DNA_sequence)
    frames['f2'] = translate_frame(DNA_sequence[1:])
    frames['f3'] = translate_frame(DNA_sequence[2:])

    # Translate three reverse frames.
    reversed_sequence = DNA_sequence[::-1]
    frames['r1'] = translate_frame(reversed_sequence)
    frames['r2'] = translate_frame(reversed_sequence[1:])
    frames['r3'] = translate_frame(reversed_sequence[2:])

    return frames

In [14]:
# Test code
_seqdic=translate("ACTGACTGACTGACTGACTGACTG")
assert type(_seqdic)==type(dict()), "Return value is not a dictionary: %r" % _seqdic
assert set(_seqdic.keys())==set(['f1', 'f2', 'f3', 'r1', 'r2', 'r3']), \
    "Output dictionary has incorrect/missing keys: %r"  % _seqdic.keys()
assert type(_seqdic['f1'])==type(""), \
    "Output dictionary values should be strings, not %r" % type(_seqdic['f1'])
print(f"Translated Frames: {_seqdic}")

Translated Frames: {'f1': 'TD*LTD*L', 'f2': 'LTD*LTD', 'f3': '*LTD*LT', 'r1': 'VSQSVSQS', 'r2': 'SVSQSVS', 'r3': 'QSVSQSV'}


### Locating an Open Reading Frame (ORF)

The openReadingFrame function identifies and returns the amino acid sequence between the first Methionine (M) and the first stop codon (*). If no start or stop codon is found, the function returns an empty string. The test ensures that the function correctly identifies the open reading frame in a sequence.

In [15]:
def openReadingFrame(aa_sequence):

    # Find the first Methionine (start_codon)
    start_codon = aa_sequence.find('M')
    
    #An empty string is returned, if the STOP codon is not located
    if start_codon == -1:
        return ''

    # Find the first STOP codon after the Methionine
    stop_codon = aa_sequence.find('*', start_codon)
    
    #An empty string is returned, if the STOP codon is not located
    if stop_codon == -1:
        return ''

    return aa_sequence[start_codon:stop_codon]

In [16]:
# Test code
_seq=openReadingFrame("AMCAPP*L")
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print(f"ORF: {_seq}")

ORF: MCAPP


### Finding the Longest ORF

The candidateProtein function scans all six reading frames of a DNA sequence to find the longest ORF. It does this by translating the sequence and using openReadingFrame to identify the ORF in each frame. The function returns the longest one. The test verifies that the function correctly identifies the longest ORF from a given DNA sequence.

In [17]:
def candidateProtein(DNA_sequence):
    # DNA sequence is translated into forward and reverse frames
    frames = translate(DNA_sequence)

    # Initialize the longest ORF as an empty string
    long_ORF = ''

    # Iterate through transalted sequences, find the longest ORF, and update long_ORF when a longer ORF is discovered
    for _, aa_sequence in frames.items():
        ORF = openReadingFrame(aa_sequence)
        if len(ORF) > len(long_ORF):
            long_ORF = ORF

    return long_ORF

In [18]:
# Test code
_seq=candidateProtein("ATGACTGCTGGGTAG")
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print(f"Longest ORF: {_seq}")

Longest ORF: MTAG


### Writing a FASTA File

The writeFASTA function writes an amino acid sequence to a file in FASTA format. It writes the provided description as the FASTA header and ensures that the sequence is formatted into lines of up to 60 characters. The test ensures that the function correctly writes the sequence to a file in the correct format.

In [19]:
def writeFASTA(sequence, description, filename):
    # Define the maximum line length.
    line_length = 60
    
    # Open the specified file in write mode using a context manager.
    with open(filename, 'w') as fasta_file:
        # Write the description line (header) in FASTA format.
        fasta_file.write(">" + description + "\n")
        
        # Iterate through the sequence, breaking it into lines of specified length.
        for i in range(0, len(sequence), line_length):
            fasta_file.write(sequence[i:i + line_length] + '\n')

In [20]:
# Test the writeFASTA function
_rv = writeFASTA(sequence="TESTTESTTESTTESTTEST", description="test sequence", filename="protein_output.fas")
assert type(_rv) is type(None), "Function should not return anything; it returns %r" % _rv
_fe = os.path.isfile("protein_output.fas")
assert _fe, "Cannot find output file - has it been created?"
print("FASTA file successfully written.")

FASTA file successfully written.


### Putting it All Together

The maximalORF function reads a DNA sequence from an input file, finds the longest ORF, and writes the resulting protein sequence to an output file in FASTA format. The header of the output file is specified by the proteinname argument. The test code confirms that the function successfully writes the longest ORF to a FASTA file.

In [21]:
def maximalORF(inputfile, outputfile, proteinname):
    # Read the DNA sequence from the input file.
    DNA_sequence = readDNAsequence(inputfile)

    # Find the candidate protein sequence with the longest open reading frame (ORF).
    protein_sequence = candidateProtein(DNA_sequence)

    # Write the candidate protein sequence in FASTA format to the output file with the provided protein name.
    writeFASTA(protein_sequence, proteinname, outputfile)

In [22]:
# Create a test DNA sequence for ORF detection
with open("dna_orf_sequence.fas", "wt") as fasta_file:
    fasta_file.write("> Example ORF Sequence\n")
    fasta_file.write("ATGACTGCTGGGTAG\n")

# Test the maximalORF function
_rv = maximalORF(inputfile="dna_orf_sequence.fas", outputfile="orf_output.fas", proteinname="test protein")
assert type(_rv) is type(None), "Function should not return anything; it returns %r" % _rv
_fe = os.path.isfile("orf_output.fas")
assert _fe, "Cannot find output file - has it been created?"
print("FASTA file written for the longest ORF.")

FASTA file written for the longest ORF.


# Section 3: Classification of aminoacids

### Reading an Amino Acid Sequence

The readAAsequence function reads an amino acid sequence from a FASTA file, discards the header, and returns the sequence as a string. If the sequence contains any invalid characters, the function raises a BadSequenceException. The test code ensures that the function reads the sequence correctly from a FASTA file.

In [23]:
def readAAsequence(fasta_file):
    # Initialize an empty string to store the amino acid sequence.
    sequence = ""
    
    # Define valid amino acids.
    valid_aminoacids = "ABCDEFGHIJKLMNOPQRSTUVWXYZ*"
    
    # Check if the specified file exists; if not, raise a FileNotFoundError.
    if not os.path.exists(fasta_file):
        raise FileNotFoundError(f"File '{fasta_file}' does not exist.")
    
    # Open the FASTA file in read mode using a context manager.
    with open(fasta_file, 'r') as FASTA:
        in_sequence = False
        for line in FASTA:
            line = line.strip()
            
            if line.startswith('>'):
                # Ignore lines starting with '>'.
                in_sequence = False
            elif in_sequence:
                # Append the current line to the sequence.
                sequence += line
            else:
                in_sequence = True
                sequence += line
        
        # Check for invalid amino acids in the sequence.
        for aminoacid in sequence:
            if aminoacid not in valid_aminoacids:
                raise BadSequenceException(f"Incorrect amino acid {aminoacid} has been detected")
    
    return sequence

In [24]:
# Create a sample amino acid FASTA file
with open("protein_sequence.fas", "wt") as fasta_file:
    fasta_file.write("> Example Protein Sequence\n")
    fasta_file.write("MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKV\n")

# Test the readAAsequence function
_seq = readAAsequence("protein_sequence.fas")
assert type(_seq) is type(""), "Return value is not a string: %r" % _seq
print(f"AA Sequence: {_seq}")

AA Sequence: MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRFFESFGDLSTPDAVMGNPKV


### Amino Acid Usage Statistics

The AAtypes function calculates the percentage of polar, small, and hydrophobic amino acids in a given sequence. Since some amino acids belong to multiple categories, the percentages may sum to more than 100%. The test ensures that the function correctly calculates these percentages for a sample sequence.

In [25]:
def AAtypes(amino_acid_sequence):
    # Define sets of amino acids in each category
    polar_amino_acids = set("YWHKREDQNSTC")  # Polar amino acids
    small_amino_acids = set("VCTSGAPDN")  # Small amino acids
    hydrophobic_amino_acids = set("ILVMFYWHKTCA")  # Hydrophobic amino acids

    # Initialize counters for each category
    polar_count = 0
    small_count = 0
    hydrophobic_count = 0

    # Iterate through the amino acid sequence and count amino acids in each category
    for aa in amino_acid_sequence:
        if aa in polar_amino_acids:
            polar_count += 1
        if aa in small_amino_acids:
            small_count += 1
        if aa in hydrophobic_amino_acids:
            hydrophobic_count += 1

    # Calculate the total length of the sequence
    total_length = len(amino_acid_sequence)

    # Calculate the fraction of each category and return as a list
    polar_fraction = polar_count / total_length
    small_fraction = small_count / total_length
    hydrophobic_fraction = hydrophobic_count / total_length

    return [polar_fraction, small_fraction, hydrophobic_fraction]

In [26]:
# Test code
_aatypes=AAtypes("TESTTESTTESTTESTTEST")
assert type(_aatypes) is type([]), "Return value is not a list: %r" % _aatypes
assert len(_aatypes)==3, "Returned list should contain 3 values"
print(f"AA Types: {_aatypes}")

AA Types: [1.0, 0.75, 0.5]


### Processing Multiple Sequences

The AAtypetable function processes multiple FASTA files, computes the percentages of polar, small, and hydrophobic amino acids for each sequence, and writes the results to a TSV file. If any files cannot be processed (due to missing data or errors), their filenames are returned in a list. The test verifies that the function processes multiple sequences and writes the results correctly to a file.

In [27]:
def AAtypetable(filelist, outputfile):
    
    with open(outputfile, 'w') as output:
        # Write the column headers
        output.write("# Filename\tPolar\tSmall\tHydro\n")

        # Initialize a list to store the names of files with errors
        error_files = []

        # Iterate through the list of filenames
        for filename in filelist:
            try:
                # Read the amino acid sequence from the file
                amino_acid_sequence = readAAsequence(filename)

                # Compute the percentages of polar, small, and hydrophobic residues
                percentages = AAtypes(amino_acid_sequence)

                # Write the results to the output file
                output.write(f"{filename}\t{percentages[0]:.2f}\t{percentages[1]:.2f}\t{percentages[2]:.2f}\n")
            except Exception as e:
                # If there's an error, append the filename to the error_files list
                error_files.append(filename)

  
    return error_files

In [28]:
# Create multiple FASTA files for processing
for fname in ["protein_1.fas", "protein_2.fas"]:
    with open(fname, "wt") as fasta_file:
        fasta_file.write("> test\n")
        fasta_file.write("TESTTESTTESTTESTTEST\n")

# Test the AAtypetable function
_rv = AAtypetable(filelist=['protein_1.fas', 'protein_2.fas'], outputfile="aa_table.txt")
_fe = os.path.isfile("aa_table.txt")
assert _fe, "Cannot find output file - has it been created?"
assert type(_rv) is type([]), "Function should return a list; it returns %r" % _rv
print(f"Table written with errors: {_rv}")

Table written with errors: []


# Section 4: Clustering proteins based on aminoacid usage

### Computing the Euclidean Distance

The distance function computes the Euclidean distance between two lists of floats, representing vectors. If the lists are not the same length, or are both empty, the function raises a DimensionalityException. The test code ensures that the function calculates the distance correctly between two vectors.

In [29]:
# Run this cell to define the exception
class DimensionalityException(Exception):
    pass

In [30]:
import math

def distance(list1, list2):
    # Check if the input lists have the same length or are both empty.
    if len(list1) != len(list2) or len(list1) == 0:
        raise DimensionalityException("Both lists are not the same length or empty")
        
    # Calculates the Euclidean distance between corresponding elements of list1 and list2, using squared differences and square root.
    squared_differences = [(x - y) ** 2 for x, y in zip(list1, list2)]
    sum_of_squares = sum(squared_differences)
    euclidean_distance = math.sqrt(sum_of_squares)
    
    return euclidean_distance

In [31]:
# Test code
_dist=distance([1.0, 2.0], [1.5, -1.0])
assert type(_dist) is type(0.0), "Return value is not a float: %r" % _dist
print(f"Distance: {_dist}")

Distance: 3.0413812651491097


### Reading Data in Tabular Format

The readTable function reads a TSV file that contains amino acid usage statistics and returns a dictionary where each key is a filename and the corresponding value is a list of percentages (polar, small, and hydrophobic amino acids). The test verifies that the function reads and parses the data correctly from a file.

In [32]:
def readTable(filename):
    # Initialize an empty dictionary to store data.
    data = {}

    with open(filename, 'r') as table_file:
        header = table_file.readline().strip().split('\t')
        
        # Iterate through the lines in the table file.
        for line in table_file:
            # Split each line into a list of values by tabs.
            values = line.strip().split('\t')
            
            # Check if there are at least four values in the line.
            if len(values) >= 4:
                # Extract the filename and convert the remaining values to floats.
                filename = values[0]
                percentages = [float(val) for val in values[1:]]

                data[filename] = percentages

    return data

In [33]:
# Create a TSV file with amino acid statistics
with open("protein_table.tsv", "wt") as tsv_file:
    tsv_file.write("# Filename\tPolar\tSmall\tHydro\n")
    tsv_file.write("Pxxx.fas\t0.52\t0.22\t0.33\n")
    tsv_file.write("Pyyy.fas\t0.47\t0.35\t0.38\n")

# Test the readTable function
_tab = readTable(filename="protein_table.tsv")
assert type(_tab) is type({}), "Return value is not a dict: %r" % _tab
assert ("Pxxx.fas" in _tab) and ("Pyyy.fas" in _tab), "Return dict missing keys"
print(f"Table Data: {_tab}")

Table Data: {'Pxxx.fas': [0.52, 0.22, 0.33], 'Pyyy.fas': [0.47, 0.35, 0.38]}


### Computing a Distance Matrix

The distanceMatrix function reads protein data from a TSV file, computes pairwise Euclidean distances between the proteins, and writes the results to an output file as a matrix. The matrix is symmetric, with zeros on the diagonal (the distance between a protein and itself is zero). The test confirms that the function correctly computes and writes the distance matrix to a file.

In [34]:
def distanceMatrix(inputfile, outputfile):
    # Read the protein data from the input file
    protein_data = readTable(inputfile)

    # Get the list of protein names from the data
    protein_names = list(protein_data.keys())

    # Initialize a square matrix filled with zeros
    distance_matrix = [[0.0] * len(protein_names) for _ in range(len(protein_names))]

    # Compute the distance matrix
    for i, protein_1 in enumerate(protein_names):
        for j, protein_2 in enumerate(protein_names):
            if j < i:
                distance_value = distance(protein_data[protein_1], protein_data[protein_2])
                distance_matrix[i][j] = distance_value
                distance_matrix[j][i] = distance_value

    # Open the output file for writing
    with open(outputfile, 'w') as output:
        # Write the column headers
        output.write("# Filename\t" + "\t".join(protein_names) + "\n")

        # Write the distance matrix with formatting
        for i, protein_name in enumerate(protein_names):
            output.write(protein_name + "\t" + "\t".join(f"{distance:.2f}" for distance in distance_matrix[i]) + "\n")

In [35]:
# Create a TSV file with amino acid statistics for the distance matrix
with open("protein_matrix.tsv", "wt") as tsv_file:
    tsv_file.write("# Filename\tPolar\tSmall\tHydro\n")
    tsv_file.write("Pxxx.fas\t0.52\t0.22\t0.33\n")
    tsv_file.write("Pyyy.fas\t0.47\t0.35\t0.38\n")

# Test the distanceMatrix function
_rv = distanceMatrix(inputfile="protein_matrix.tsv", outputfile="distance_matrix.tsv")
_fe = os.path.isfile("distance_matrix.tsv")
assert _fe, "Cannot find output file - has it been created?"
with open("distance_matrix.tsv", "rt") as inf:
    _fl = inf.readline()
assert _fl[0] == '#', "First line of output file does not start with a #"
print("Distance matrix successfully written.")

Distance matrix successfully written.
